In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [27]:
data = pd.read_excel("Bank_Personal_Loan_Modelling.xlsx",sheet_name=1)

In [28]:
# Drop irrelevant columns like ID and ZIP Code
data.drop(['ID', 'ZIP Code'], axis=1, inplace=True)

# Convert categorical variables to dummy variables
data = pd.get_dummies(data, columns=['Family', 'Education', 'Securities Account',
                                     'CD Account', 'Online', 'CreditCard'])

# Split the data into features (X) and target (y)
X = data.drop('Personal Loan', axis=1)
y = data['Personal Loan']

# Split the data into training and testing sets
np.random.seed(42)  # For reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bernoulli Naive Bayes Classifier Implementation
class BernoulliNaiveBayes:
    def fit(self, X, y):
        self.num_classes = np.unique(y)
        self.prior = {}
        self.likelihood = {}
        
        for c in self.num_classes:
            X_c = X[y == c]
            self.prior[c] = len(X_c) / len(X)
            self.likelihood[c] = (X_c.sum(axis=0) + 1) / (len(X_c) + 2)  # Laplace smoothing
        
    def predict(self, X):
        predictions = []
        
        for _, row in X.iterrows():
            probs = {c: np.log(self.prior[c]) + np.sum(np.log(self.likelihood[c]) * row) for c in self.num_classes}
            prediction = max(probs, key=probs.get)
            predictions.append(prediction)
        
        return np.array(predictions)

# Multinomial Naive Bayes Classifier Implementation
class MultinomialNaiveBayes:
    def fit(self, X, y):
        self.num_classes = np.unique(y)
        self.prior = {}
        self.likelihood = {}
        
        for c in self.num_classes:
            X_c = X[y == c]
            self.prior[c] = len(X_c) / len(X)
            self.likelihood[c] = (X_c.sum(axis=0) + 1) / (len(X_c) + 2)  # Laplace smoothing
        
    def predict(self, X):
        predictions = []
        
        for _, row in X.iterrows():
            probs = {c: np.log(self.prior[c]) + np.sum(np.log(self.likelihood[c]) * row) for c in self.num_classes}
            prediction = max(probs, key=probs.get)
            predictions.append(prediction)
        
        return np.array(predictions)

# Gaussian Naive Bayes Classifier Implementation
class GaussianNaiveBayes:
    def fit(self, X, y):
        self.num_classes = np.unique(y)
        self.mean = {}
        self.var = {}
        self.prior = {}
        
        for c in self.num_classes:
            X_c = X[y == c]
            self.mean[c] = np.mean(X_c, axis=0)
            self.var[c] = np.var(X_c, axis=0)
            self.prior[c] = len(X_c) / len(X)
        
    def predict(self, X):
        predictions = []
        
        for _, row in X.iterrows():
            probs = {c: np.log(self.prior[c]) - 0.5 * np.sum(np.log(2 * np.pi * self.var[c])) \
                            - 0.5 * np.sum(((row - self.mean[c]) ** 2) / (self.var[c])) \
                     for c in self.num_classes}
            prediction = max(probs, key=probs.get)
            predictions.append(prediction)
        
        return np.array(predictions)

# Instantiate the classifiers
bnb = BernoulliNaiveBayes()
mnb = MultinomialNaiveBayes()
gnb = GaussianNaiveBayes()

# Train the classifiers
bnb.fit(X_train, y_train)
mnb.fit(X_train, y_train)
gnb.fit(X_train, y_train)

# Predict on the test set
y_pred_bnb = bnb.predict(X_test)
y_pred_mnb = mnb.predict(X_test)
y_pred_gnb = gnb.predict(X_test)

# Calculate Precision, Recall, and F1-score for each classifier
def calculate_metrics(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    return precision, recall, f1_score

precision_bnb, recall_bnb, f1_score_bnb = calculate_metrics(y_test, y_pred_bnb)
precision_mnb, recall_mnb, f1_score_mnb = calculate_metrics(y_test, y_pred_mnb)
precision_gnb, recall_gnb, f1_score_gnb = calculate_metrics(y_test, y_pred_gnb)

print("Bernoulli Naive Bayes:")
print("Precision:", precision_bnb)
print("Recall:", recall_bnb)
print("F1-Score:", f1_score_bnb)
print()

print("Multinomial Naive Bayes:")
print("Precision:", precision_mnb)
print("Recall:", recall_mnb)
print("F1-Score:", f1_score_mnb)
print()

print("Gaussian Naive Bayes:")
print("Precision:", precision_gnb)
print("Recall:", recall_gnb)
print("F1-Score:", f1_score_gnb)

Bernoulli Naive Bayes:
Precision: 0.105
Recall: 1.0
F1-Score: 0.19004524886877827

Multinomial Naive Bayes:
Precision: 0.105
Recall: 1.0
F1-Score: 0.19004524886877827

Gaussian Naive Bayes:
Precision: 0.5588235294117647
Recall: 0.5428571428571428
F1-Score: 0.5507246376811593
